<h2> Spectroscopic data reduction : spectra module

In [2]:
from pyvista import imred, tv, spectra
import pyvista.data
from importlib_resources import files
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

pyvista uses a display tool defined in the tv module. To use the interactive
display in a notebook, set the display to be an external display window, e.g. with 
<code>
%matplotlib qt
</code>
Instantiate a tv object, here we just call it t, but you could call it whatever you want!

In [ ]:
%matplotlib qt
t=tv.TV()

The basic tool for basic image reduction is a Reducer object, defined in the imred module. Instantiate a reducer here. The main argument is an instrument name, which tells it to read a YAML configuration file for the specified instrument. We also give it an optional dir= argument to specify the default directory from which to read images, if a directory is not specified in subsequent commands that read images.

In [ ]:
indir='/home/users/adijeau/Documents/UT220107'
red=imred.Reducer('KOSMOS',dir=indir)

A main method of the reducer object is the reduce() method. Without any additional arguments, reduce() will read an image from disk, subtract the overscan (region(s) as determined from the instrument configuration file), compute an uncertainty array using the gain and readout noise from the instrument configuration file, and return a CCDData object with the data, uncertainty, and mask. 
<p>
To specify the input image, we could pass a string with the file name. If the string does not include a '/', it will read from the default input directory.
<p>
If the file can be identified with a unique integer, then you can just specify this number, which can be very convenient. This is turned into a character string using the formstr attribute define in the configuration file, which is used to search for the file to read.
<p>
We can display the image using the tv() method of our display tool, which can take as input a CCDData object, and numpy array, or a FITS HDU object.

In [ ]:
a=red.reduce(22)

t.tv(a)

<h4> Calibration: make and apply flat field

If we add additional arguments to reduce(), we can add additional calibration steps. For example, to flat field the data, we would add a flat= keyword through which we give the reducer a flat field.
<br>
First, however, we have to make the flat field, which is accomplished using the mkflat() method, which takes as input a list of frames to be used to construct the master flat field (superflat). For a spectrograph, we use the spec=True keyword which will remove the spectral signature from the combined flat.

In [ ]:
flatims=[61,62,63,64,65]
flat=red.mkflat(flatims,spec=True,display=None)

Read and display a star spectral image

In [ ]:
star=red.reduce(2,flat=flat)
t.tv(star,max=1000) 

In [ ]:
crstar=red.crrej(star,crbox=[9,1],display=t)

In [ ]:
t.tv(crstar.bitmask,min=0,max=2)


<h4> Tracing and extraction

We want to extract the spectrum, i.e. from the 2D image to a 1D spectrum. Start by defining a trace that is just constant along rows, and extract the spectrum with a 25 pixel window.

In [ ]:
# here we set a trace to be at a constant position on the detector
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
vars(trace) 

We can use the existing trace to retrace a new spectrum. This is done by cross-correlating the reference spectrum with a spatial cut across the input spectrum (at sc0) to find the shift, then using the old model to start the trace from that starting position. The functional form of the old model is preserved:

In [ ]:
trace.retrace(star,display=t)

Alternatively, one could use the find() method to determine the shift between the stored trace and your frame automatically with cross correlation, then just go straight to extraction using the shape of the stored trace; this would be applicable if your object is too faint, or doesn't have sufficient continuum, to trace. If you want to mark the location of the object manually, use the inter=True keyword (along with display=) in find(). 


In [ ]:
trace.model[0](2048)
trace.find(star,inter=True,display=t)
print(trace.pix0)

You can also just find peak(s) in the spatial direction using findpeak(), then trace them:

In [ ]:
peaks,fibers=trace.findpeak(star,thresh=5)
print(peaks)
print(fibers)
                            
trace.trace(star,peaks,display=t)

The model is now modified for the input spectrum:

In [ ]:
trace.model[0](2048)

If you needed to create Trace from scratch, see below. Note the skip= keyword which can be used when tracing to speed things up by only computing centroids every skip pixels, taking a median around these pixels of width skip pixels. The default is skip=10

In [ ]:
trace=spectra.Trace(sc0=2048,lags=range(-300,300),
                    rows=[550,1450],transpose=red.transpose)
vars(trace)
srow=[955]   # list to allow for multiple spectra on an image
starec=trace.trace(star,srow,display=t,skip=20) 
vars(trace)

Extraction is done using the extract() method of the Trace object, currently just with simple boxcar extraction. Use rad= to specify window radius. Use back=[[b1,b2],[b3,b4]] to subtract background as determined from one or more background windows (note argument should be list of 2-element lists, giving start and end pixel of each background window).

In [ ]:
starec=trace.extract(star,display=t,rad=25)
print(starec.shape)

extract() returns a CCDData object, with extracted spectrum, and uncertainty. Note that this is a 2D array to accomodate multiple objects/traces, even if there is only a single object.

In [ ]:
plt.figure()
plt.plot(starec.data[0])
plt.plot(starec.data[0]/starec.uncertainty.array[0])

In [ ]:
star.data+=5000
starec2=trace.extract(star,rad=25,back=[[50,75],[-75,-50]],display=t)
plt.figure()
plt.plot(starec.data[0])
plt.plot(starec2.data[0])
plt.show()

<h4> Wavelength calibration

Now let's turn to wavelength calibration, i.e. getting a function that gives the wavelength as a function of pixel. We'll solve for this using arc frames, here taken with each lamp separately, so sum the three exposures

In [ ]:
#Frame 15 is He, 16 is Ne, and 17 is Ar
arcs=red.sum([46,47,48])
t.clear()
t.tv(arcs)

In [ ]:
arcec=trace.extract(arcs,display=None,rad=20)

Wavelength calibration first stars with identifying lines. This is much easier if one can work from a previous solution. Here we start by reading a previous solution into a pyvista WaveCal object.

In [5]:
wav=spectra.WaveCal('KOSMOS/KOSMOS_red_waves.fits')
vars(wav)

  rms:    0.177 Angstroms (50 lines)


1

With a previous solution loaded, the identify routine will cross correlate the input spectrum with the previous solution, then attempt to identify the lines from the previous solution at the shifted pixel position from the previous solution. Finally, it does a fit.

The identify() method fits Gaussians to the lines to get their position, and gets FWHM in the process. rad= keyword specifies number of pixels on either side of line to use in fit. If you add the file= keyword, then the routine will try to identify all of the lines from the reference file; this might be useful if you wanted to extend the wavelength range of the lines from the initial object, e.g. if you were working at a different grating tilt or slit location. If you did that, you might want to save your WaveCal object after cleaning the line list for good lines, so you could use that as a starting guess for other data taken at a similar wavelength setting.

In [ ]:
wav.identify(arcec,plot=True,rad=10) #,file='henearkr.dat')

Note that once you remove lines, they are removed for subsequent uses of the WaveCal object.

If you want to refit with existing line positions, e.g. to try a different model, you can just call fit().

In [ ]:
wav.fit(degree=5)

Show the FWHM of the lines as a function of wavelength

In [ ]:
wav.fwhm
#t.plotax2.cla()
#t.plotax2.plot(wav.waves,np.abs(wav.fwhm),'ro')

OK, now use the wavelength solution to get wavelength as a function of pixel. We can save that in the wave attribute of our Data object.

In [ ]:
wav.add_wave(starec)
plt.figure()
plt.plot(starec.wave[0],starec.data[0])

# get inverse relation, i.e. pixels as f(wavelength)
pix=np.arange(4096)
from scipy.interpolate import CubicSpline
wav2pix=CubicSpline(np.flip(starec.wave[0]),np.flip(pix))

Next cell has resampling of spectrum onto a uniform wavelength grid

In [ ]:
plt.figure()
plt.clf()
plt.plot(starec.wave[0],starec.data[0])
print(1/(5.5e-5*np.log(10)))
wnew=10**np.arange(3.5,4.0,5.5e-6)
plt.plot(wnew,wav.scomb(starec,wnew).data)

<h4> Adjusting wavelength solution for flexure

Next cell shows some possibilities for adjusting wavelength solution based on sky lines

In [ ]:
objec=trace.extract(red.reduce(22),display=None,rad=20)
wav.add_wave(objec)


In [ ]:
plt.figure()
plt.plot(objec.wave[0],objec.data[0])

If we fix all but the first term, we solve for a constant wavelength shift

In [ ]:
import copy
swav=copy.copy(wav)
wave=wav.wave(image=objec.shape)
swav.model.fixed['c1']=True
swav.model.fixed['c2']=True
swav.model.fixed['c3']=True 
swav.identify(objec,wav=objec.wave,file='skyline.dat',plot=True,thresh=50,inter=True)
print(wav.model)
print(swav.model)

<h4> Flux calibration

Instantiate a FluxCal object. You can set it up to use a polynomial fit to the response curve by specifying the polynomial degree with the degree= keyword. If you set degree=-1, the response curve will be determined by a median of the individual response curves, optionally smoothed with a median filter over wavelength (see response() method below).

In [ ]:
flx=spectra.FluxCal(degree=-1)
polyflx=spectra.FluxCal(degree=5)

Load in several extracted spectra of flux standards, here all of Feige 34. The standard star spectrum is given using the file= keyword, where the input file should have labelled columns wave, flux, and bin. Alternatively, you can pass an astropy Table with (at least) these three columns, using the stdflux= keyword

In [ ]:
for im in [2,3, 38,39] :
    star=red.reduce(im)
    trace.retrace(star)
    starec=trace.extract(star)
    wav.add_wave(starec)
    flx.addstar(starec[0],starec.wave[0],file='flux/okestan/ffeige34.dat')
    polyflx.addstar(starec[0],starec.wave[0],file='flux/okestan/ffeige34.dat')

Solve for the median response curve, taking a median filter over this median.

In [ ]:
flx.response(plot=True,medfilt=200)
polyflx.response(plot=True)

Apply the response curve to an object. Note that, using Data objects, the S/N is preserved!

In [ ]:
fig,ax=plots.multi(1,2,hspace=0.001)
ax[0].plot(starec.wave[0],starec.data[0])
ax[1].plot(waves,starec.data[0]/starec.uncertainty.array[0])
flx.correct(starec,starec.wave)
ax[0].plot(starec.wave[0],starec.data[0])
ax[1].plot(waves,starec.data[0]/starec.uncertainty.array[0])

<h3> longslit extraction and wavelength calibration

For extended objects, and perhaps for sky subtraction, we might want to work along the slit. The wavelength solution varies along the slit (line curvature), usually with more than just an offset.

Start by working along the slit to identify lines

In [ ]:
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
arc2d=trace.extract2d(arcs)
t=tv.TV()
t.tv(arc2d)

In [ ]:
from pyvista import image
wav=spectra.WaveCal(file='KOSMOS/KOSMOS_red_waves.fits')
image.smooth(arc2d,[5,1])
t.clear()
t.tv(arc2d)
wav.identify(arc2d, rad=10,display=None, plot=True,
              nskip=20,lags=np.arange(-10,10))
wav.add_wave(arc2d)

OK, use the solution to make a wavelength map

In [ ]:
t.clear()
t.tv(arc2d.wave)

Subtract out the central wavelength solution to see how the solution varies with row

In [ ]:
dw=arc2d.wave-arc2d.wave[450]
t.tv(dw)  

In [ ]:
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
obj=red.reduce(22)
t.tv(obj)
obj2d=trace.extract2d(obj)
wav.add_wave(obj2d)
t.tv(obj2d)
star=red.reduce(38)
star2d=trace.extract2d(star)
wav.add_wave(star2d)

Here we rectify the image to have a constant wavelength scale (in log lambda). We choose the new scale based on the starting and ending wavelengths in the original image, and resample to get the same number of pixels.

If you preferred not to resample your object (or at least minimize the ressampling), you could resample the image to the wavelength array at the location of your object

In [ ]:
wlim=(arc2d.wave[450,0],arc2d.wave[450,-1])
wnew=10.**np.linspace(np.log10(wlim[1]),np.log10(wlim[0]),4095)
print(wnew)
arc2d_rect=wav.correct(arc2d,wnew)
t.tv(arc2d)
t.tv(arc2d_rect)

In [ ]:
t.tv(obj2d)
obj2d_rect=wav.correct(obj2d,wnew)
star2d_rect=wav.correct(star2d,wnew)
t.tv(obj2d_rect)


OK, now we can extract in the wavelength rectified image for better sky subtraction. Create a new trace object with the rows= attribute to give the new extent of the image in rows, since our previous extraction limited the extraction to the length of the slit, as set by the rows attribute in the original trace object.

In [ ]:
rows=[0,star2d_rect.shape[0]]
trace=spectra.Trace(sc0=int(star2d_rect.shape[1]/2.),rows=rows)
t.clear()
rows,fibers=trace.findpeak(star2d_rect)
print('rows: ',rows)
trace.trace(star2d_rect,rows,display=t)
ext=trace.extract(obj2d_rect,display=t,rad=10)
ext.add_wave(wnew)
ext_sub=trace.extract(obj2d_rect,rad=10,back=[[20,30]],display=t)
ext_sub.add_wave(wnew)

In [ ]:
plt.figure()
plt.plot(ext.wave,ext.data[0])
plt.plot(ext.wave,ext_sub.data[0])

CR rejection via multiple image stacking

<h4> KOSMOS blue channel

In [ ]:
bwav=spectra.WaveCal('KOSMOS/KOSMOS_blue_waves.fits')
trace=spectra.Trace('KOSMOS/KOSMOS_trace.fits')
arcs=red.sum([49,50,51])
arcec=trace.extract(arcs)
bwav.identify(arcec,plot=True)


In [ ]:
objec=trace.extract(red.reduce(26),display=None,rad=20)
bwav.add_wave(objec)
plt.figure()
plt.plot(objec.wave[0],objec.data[0])

swav=copy.copy(bwav)
swav.model.fixed['c0']=False
swav.model.fixed['c1']=True
swav.model.fixed['c2']=True
swav.model.fixed['c3']=True 
swav.identify(objec,wav=objec.wave,file='skyline.dat',plot=True,thresh=50,inter=True)
print(bwav.model)
print(swav.model)

In [ ]:
flx=spectra.FluxCal(degree=5)
for im in [34,35,36] :
    star=red.reduce(im)
    trace.retrace(star)
    starec=trace.extract(star)
    bwav.add_wave(starec)
    flx.addstar(starec[0],wave[0],file='flux/okestan/ffeige34.dat')

In [ ]:
flx.response(plot=True)

In [ ]:
plt.figure()
flx.correct(starec,starec.wave[0])
plt.plot(starec.wave[0],starec.data[0])